In [2]:
import random
import torch
from torch.autograd import Variable
from tensorboard import SummaryWriter


writer = SummaryWriter()

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])
    if t % 10 == 0:
        for name, param in model.named_parameters():
            writer.add_histogram(name, param.clone().data.numpy(), t)


    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
z = model(Variable(torch.randn(N, D_in), requires_grad=True))
writer.add_graph(model, z)
writer.close()

(0, 655.845703125)
(1, 645.8165283203125)
(2, 643.9903564453125)
(3, 641.5662231445312)
(4, 618.60302734375)
(5, 668.018798828125)
(6, 634.15087890625)
(7, 581.1384887695312)
(8, 626.7577514648438)
(9, 622.8908081054688)
(10, 491.16748046875)
(11, 542.280029296875)
(12, 413.54913330078125)
(13, 364.7979431152344)
(14, 607.9328002929688)
(15, 511.7772521972656)
(16, 497.8021545410156)
(17, 476.31610107421875)
(18, 177.06961059570312)
(19, 616.2239990234375)
(20, 396.9598693847656)
(21, 116.53943634033203)
(22, 549.65673828125)
(23, 598.0808715820312)
(24, 511.5323181152344)
(25, 268.61602783203125)
(26, 549.1695556640625)
(27, 422.04449462890625)
(28, 385.9467468261719)
(29, 450.37371826171875)
(30, 322.569580078125)
(31, 259.47833251953125)
(32, 295.98577880859375)
(33, 342.7735900878906)
(34, 221.04718017578125)
(35, 178.86570739746094)
(36, 272.4000244140625)
(37, 127.98677825927734)
(38, 254.71539306640625)
(39, 90.92464447021484)
(40, 224.15972900390625)
(41, 98.35035705566406)
(42